In [ ]:
import torch
from torch.autograd import Variable
from torchvision import datasets, transforms
import numpy as np
import boto3
import requests
from io import BytesIO
from datetime import datetime
import json
from pytz import timezone
import matplotlib.pyplot as plt

In [ ]:
s3 = boto3.resource('s3')
bucket = "attendance-model"

In [ ]:
def get_model(model_path):

    if model_path[:5] == 's3://':
        s3_bucket, s3_key = model_path.split('/')[2], model_path.split('/')[3:]
        s3_key = '/'.join(s3_key)
        with BytesIO() as f:
            boto3.client("s3").download_fileobj(Bucket=s3_bucket, Key=s3_key, Fileobj=f)
            f.seek(0)
            model_file = torch.load(f)
    else:
        with open(model_path, 'rb') as f:
            model_file = torch.load(f)
    return model_file

In [ ]:
def imshow(input, title):
    input = input.cpu().numpy().transpose((1, 2, 0))
    input = std_arr * input + mean_arr
    input = np.clip(input, 0, 1)

    plt.figure(figsize=(12, 6))
    plt.title(title)
    plt.imshow(input)
    plt.pause(2)

In [ ]:
with torch.no_grad():
    model_path = get_model('s3://attendance-model/model_final')
    total = 0
    correct = 0
    arr = [['1131901', 'Durvesh'], ['1149695', 'Frances'], ['1146105', 'Jay'], ['1138948', 'Kamal'], ['1138765', 'yash']]

    mean_arr = [0.485, 0.456, 0.406]
    std_arr = [0.229, 0.224, 0.225]

    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean_arr,
                             std=std_arr)
    ])

    img = s3.Bucket(bucket).download_file('image.jpg', 'local_img.jpg')
    value = transforms.ToTensor()
    transformed_img = transform(img).float()
    transformed_img = Variable(transformed_img, requires_grad=True)
    transformed_img = transformed_img.unsqueeze(0)
    transformed_img = transformed_img
    result = model_path(transformed_img)
    _, predicted_output = torch.max(result.data, 1)
    predicted_index = predicted_output.cpu().numpy()[0]
    imshow(value(img), arr[predicted_index])

In [ ]:
api_get = 'https://1o739743u5.execute-api.us-east-1.amazonaws.com/getV1/putdyanamodb?id=' + \
          arr[predicted_index][0]

res = requests.get(api_get)
json_data = res.json()

api_post = "https://1o739743u5.execute-api.us-east-1.amazonaws.com/v1/putdyanamodb"

array_Count = json_data["Count"]

fmt = '%Y-%m-%d %H:%M:%S'
eastern = timezone('US/Eastern')
loc_dt = datetime.now(eastern)

current_date = loc_dt.strftime(fmt)

att = [
    str(current_date)
]

str1 = ','.join(str(e) for e in att)

if array_Count == 1:
    s = json_data["Items"][0]
    password = s["password"]["S"]
    last_name = s["last_name"]["S"]
    attendance = s["attendance"]["S"]
    first_name = s["first_name"]["S"]
    id = s["id"]["S"]
    email = s["email"]["S"]

    concat_attendance = attendance + ',' + str1

    string = {
        "id": id,
        "first_name": first_name,
        "last_name": last_name,
        "email": email,
        "password": password,
        "attendance": concat_attendance
    }

    string = json.dumps(string)
    res = requests.post(api_post, data=string)
    print('Data Successfully Created ', res.text)